# Agents and Tools

#### ChatGPT Functions

In [54]:
import sqlite3
conn = sqlite3.connect('db.sqlite')

In [55]:
def run_query(query:str):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        return cursor.fetchall()
    except sqlite3.OperationalError as err:
        return f"The following error occurred: {str(err)}"
    
def describe_tables(tables_names):
    c = conn.cursor()
    tables = ', '.join("'" + table + "'" for table in tables_names)
    query = f"SELECT sql FROM sqlite_master WHERE type='table' and name in ({tables});"
    rows = c.execute(query) 
    formated = '\n'.join(row[0] for row in rows if row[0] is not None)
    return formated



## Making Custom Schema

In [56]:
from pydantic.v1 import BaseModel

class RunQueryArgsSchema(BaseModel):
    query: str

class DescribeTablesArgsSchema(BaseModel):
    tables_names: list[str]

In [57]:
from langchain.tools import Tool


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="""
    run a query on the sqlite database
    """,
    func=run_query,
    args_schema=RunQueryArgsSchema,
)

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description="""
    Given a list of tables name, return the tables that are present in the database
    """,
    func=describe_tables,
    args_schema=DescribeTablesArgsSchema,
)

In [58]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from dotenv import load_dotenv
from langchain.schema import SystemMessage
load_dotenv()


True

In [59]:
def list_list_tables():
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall() 
    return "\n".join(row[0] for row in rows if row[0] is not None)


## Callbacks

In [60]:
from langchain.callbacks.base import BaseCallbackHandler
from pyboxen import boxen

In [61]:
def boxen_print(*args, **kwargs):
    print(boxen(*args, **kwargs))
    print("\n")

In [62]:
class ChatModelStartHandler(BaseCallbackHandler):

    def on_chat_model_start(
            self,
            serialized,
            messages,
            **kwargs
    ):
        
        for message in messages[0]:
            if message.type == "system":
                boxen_print(
                    message.content, title="System", 
                    color="yellow"
                )

            elif message.type == "human":
                boxen_print(
                    message.content, title="Human", 
                    color="blue"
                )
            
            elif message.type == "ai" and "function_call" in message.additional_kwargs:
                call = message.additional_kwargs["function_call"]

                boxen_print(
                    f"Running tool {call['name']} with args {call['arguments']}",
                    title=message.type,
                    color="cyan"
                )

            elif message.type == "ai":
                boxen_print(
                    message.content, 
                    title="AI", 
                    color="red"
                )

            elif message.type == "function":
                boxen_print(
                    message.content, title="Function", color="green"
                )

            else:
                boxen_print(
                    message.content, 
                    title=message.type, color="magenta"
                )

        # return {
        #     "chat_history": messages
        # }

In [63]:
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
    callbacks=[
        ChatModelStartHandler()
    ]
)

tables = list_list_tables()



chat_prompt = ChatPromptTemplate(
    messages =[
        SystemMessage(
            content=f"You are an AI that has access to SQLite database.\n The database has tables of:\n {tables}.\n Do not make any assumptions about the what tables exist or what colums exist. Instead use the 'describe_tables' function."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [64]:
tools = [run_query_tool, describe_tables_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=tools,
)

agent_executor = AgentExecutor(
    agent=agent,
    # verbose=True,
    tools=tools,
)


In [65]:
agent_executor("How many users are in the database?")

╭─ System ─────────────────────────────────────────────────────────────────────╮
│You are an AI that has access to SQLite database.                             │
│ The database has tables of:                                                  │
│ users                                                                        │
│addresses                                                                     │
│products                                                                      │
│carts                                                                         │
│orders                                                                        │
│order_products.                                                               │
│ Do not make any assumptions about the what tables exist or what colums exist.│
│Instead use the 'describe_tables' function.                                   │
╰──────────────────────────────────────────────────────────────────────────────╯



╭─ Human ────────────────

{'input': 'How many users are in the database?',
 'output': 'There are 2000 users in the database.'}

In [66]:
agent_executor("How many users provided shipping address?")

╭─ System ─────────────────────────────────────────────────────────────────────╮
│You are an AI that has access to SQLite database.                             │
│ The database has tables of:                                                  │
│ users                                                                        │
│addresses                                                                     │
│products                                                                      │
│carts                                                                         │
│orders                                                                        │
│order_products.                                                               │
│ Do not make any assumptions about the what tables exist or what colums exist.│
│Instead use the 'describe_tables' function.                                   │
╰──────────────────────────────────────────────────────────────────────────────╯



╭─ Human ────────────────

{'input': 'How many users provided shipping address?',
 'output': 'There are 2000 users who have provided a shipping address.'}

## Report maker

In [67]:
from langchain.tools import StructuredTool

In [68]:
class ReportHTMLArgsSchema(BaseModel):
    filename: str
    html: str

In [69]:
def write_report_html(filename:str, html:str):
    with open(filename, "w") as f:
        f.write(html)
    return f"Report saved to {filename}"

In [70]:
write_report_tool = StructuredTool.from_function(
    name="write_report",
    description="Write and HTML file to disk. use this tool whenever asked for a report",
    func=write_report_html,
    args_schema=ReportHTMLArgsSchema
)

In [71]:
tools_with_html = tools + [write_report_tool]


html_agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=tools_with_html,
)

html_agent_executor = AgentExecutor(
    agent=html_agent,
    verbose=True,
    tools=tools_with_html,
)



In [72]:

html_agent_executor("Summerize top 5 most popular products and save the report to 'report.html'")



> Entering new AgentExecutor chain...
╭─ System ─────────────────────────────────────────────────────────────────────╮
│You are an AI that has access to SQLite database.                             │
│ The database has tables of:                                                  │
│ users                                                                        │
│addresses                                                                     │
│products                                                                      │
│carts                                                                         │
│orders                                                                        │
│order_products.                                                               │
│ Do not make any assumptions about the what tables exist or what colums exist.│
│Instead use the 'describe_tables' function.                                   │
╰────────────────────────────────────────────────────────────────────

{'input': "Summerize top 5 most popular products and save the report to 'report.html'",
 'output': "I have summarized the top 5 most popular products and saved the report to 'report.html'. You can download the report [here](sandbox:/report.html)."}

## Agent Scratchpad vs Memory

In [73]:
from langchain.memory import ConversationBufferMemory

In [74]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [75]:
memory_prompt = ChatPromptTemplate(
    messages =[
        SystemMessage(
            content=f"You are an AI that has access to SQLite database.\n The database has tables of:\n {tables}.\n Do not make any assumptions about the what tables exist or what colums exist. Instead use the 'describe_tables' function."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
  
    ]
)

In [76]:
memory_agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=memory_prompt,
    tools=tools_with_html,
    memory=memory
)

In [77]:
memory_agent_executor = AgentExecutor(
    agent=memory_agent,
    verbose=True,
    tools=tools_with_html,
    memory=memory
)

In [78]:
memory_agent_executor("How many users are in the database?")




> Entering new AgentExecutor chain...
╭─ System ─────────────────────────────────────────────────────────────────────╮
│You are an AI that has access to SQLite database.                             │
│ The database has tables of:                                                  │
│ users                                                                        │
│addresses                                                                     │
│products                                                                      │
│carts                                                                         │
│orders                                                                        │
│order_products.                                                               │
│ Do not make any assumptions about the what tables exist or what colums exist.│
│Instead use the 'describe_tables' function.                                   │
╰────────────────────────────────────────────────────────────────────

{'input': 'How many users are in the database?',
 'chat_history': [HumanMessage(content='How many users are in the database?'),
  AIMessage(content='There are 2000 users in the database.')],
 'output': 'There are 2000 users in the database.'}

In [79]:
memory_agent_executor("Repeat the exact same process for orders table")



> Entering new AgentExecutor chain...
╭─ System ─────────────────────────────────────────────────────────────────────╮
│You are an AI that has access to SQLite database.                             │
│ The database has tables of:                                                  │
│ users                                                                        │
│addresses                                                                     │
│products                                                                      │
│carts                                                                         │
│orders                                                                        │
│order_products.                                                               │
│ Do not make any assumptions about the what tables exist or what colums exist.│
│Instead use the 'describe_tables' function.                                   │
╰────────────────────────────────────────────────────────────────────

{'input': 'Repeat the exact same process for orders table',
 'chat_history': [HumanMessage(content='How many users are in the database?'),
  AIMessage(content='There are 2000 users in the database.'),
  HumanMessage(content='Repeat the exact same process for orders table'),
  AIMessage(content='There are 5000 orders in the database.')],
 'output': 'There are 5000 orders in the database.'}